In [17]:
import io
import re
import glob
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_districts_in_India")
s = BeautifulSoup(r.content, 'html.parser')

In [4]:
tables = s.find_all('table')
len(tables)

54

In [5]:
a = -1
tableData = []
for i in tables:
    a+=1
    print(a)
    htmlData = str(i)
    txtData = re.sub(r"\[.*?\]", "", htmlData)
    df = pd.read_html(io.StringIO(txtData))[0]
    print(df.head())
    tableData.append(df)

0
  Districts of India                               Districts of India.1
0                NaN                                                NaN
1           Category         second/third-level administrative division
2           Location              States and union territories of India
3             Number                       797 (as of 20 November 2023)
4        Populations  Greatest: Thane, Maharashtra—11,060,148 (2011 ...
1
  Administrative divisions No. of districts Total population  \
                    States           States           States   
0           Andhra Pradesh               26         49577103   
1        Arunachal Pradesh               26          1383727   
2                    Assam               35         31205576   
3                    Bihar               38        104099452   
4             Chhattisgarh               33         25545198   

  Population per district Unnamed: 4_level_0 Unnamed: 5_level_0  \
                   States             States    

In [6]:
tableData[0]

,Districts of India,Districts of India.1
0,NaN,NaN
1,Category,second/third-level administrative division
2,Location,States and union territories of India
3,Number,797 (as of 20 November 2023)
4,Populations,"Greatest: Thane, Maharashtra—11,060,148 (2011 ..."
5,Areas,"Largest: Kutch, Gujarat—45,652 km2 (17,626 sq ..."
6,Densities,"Largest: Central Delhi, Delhi Smallest: Lower ..."
7,Government,District Administration
8,Subdivisions,"Revenue division (Sub-division) and,Tehsil (Ta..."


In [7]:
statesDf = tableData[1]
statesDf = statesDf.drop(28).drop(37)
statesDf

,Administrative divisions,No. of districts,Total population,Population per district,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0
,States,States,States,States,States,States,States,States,States
0,Andhra Pradesh,26,49577103,1906812,NaN,NaN,NaN,NaN,NaN
1,Arunachal Pradesh,26,1383727,53220,NaN,NaN,NaN,NaN,NaN
2,Assam,35,31205576,891588,NaN,NaN,NaN,NaN,NaN
3,Bihar,38,104099452,2739459,NaN,NaN,NaN,NaN,NaN
4,Chhattisgarh,33,25545198,774097,NaN,NaN,NaN,NaN,NaN
5,Goa,2,1458545,729273,NaN,NaN,NaN,NaN,NaN
6,Gujarat,33,60439692,1831506,NaN,NaN,NaN,NaN,NaN
7,Haryana,22,25351462,1152339,NaN,NaN,NaN,NaN,NaN
8,Himachal Pradesh,12,6864602,528046,NaN,NaN,NaN,NaN,NaN


In [8]:
states = sum(statesDf['Administrative divisions'].values.tolist(),[])
states

['Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chhattisgarh',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal',
 'Andaman and Nicobar Islands',
 'Chandigarh',
 'Dadra Nagar Haveli, Daman and Diu',
 'Jammu and Kashmir',
 'Ladakh',
 'Lakshadweep',
 'Delhi',
 'Puducherry']

In [9]:
[x for x in tableData[3].columns if 'Population' in x]

['Population (2021)']

In [13]:
stateNo = 3
for i in states:
    df = tableData[stateNo]
    colList = [x for x in df.columns if 'Population' in x]
    df = df.rename(columns={colList[0]: 'Population'})
    df['State/UnionTerritory']=i
    df['City Tier'] = np.nan
    df['Population'] = df['Population'].replace('–', 0).replace('-', 0)
    df['Population'] = pd.to_numeric(df['Population'])
    tier1Mask = df['Population'] >= 100000
    tier2Mask = (df['Population'] <= 99999) & (df['Population'] >= 50000)
    tier3Mask = df['Population'] <= 49999
    df['City Tier'][tier1Mask] = 'Tier I'
    df['City Tier'][tier2Mask] = 'Tier II'
    df['City Tier'][tier3Mask] = 'Tier III'
    df.to_csv('States/'+i+'.csv', index = False)
    stateNo+=1

/var/folders/hz/1tq14hps0cvgb6gmb21yyrnw0000gn/T/ipykernel_56156/3476055103.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['City Tier'][tier1Mask] = 'Tier I'
/var/folders/hz/1tq14hps0cvgb6gmb21yyrnw0000gn/T/ipykernel_56156/3476055103.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Tier I' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['City Tier'][tier1Mask] = 'Tier I'
/var/folders/hz/1tq14hps0cvgb6gmb21yyrnw0000gn/T/ipykernel_56156/3476055103.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [47]:
x = glob.glob("States//*.csv")
x

['States/Nagaland.csv',
 'States/Punjab.csv',
 'States/Uttarakhand.csv',
 'States/Dadra Nagar Haveli, Daman and Diu.csv',
 'States/Delhi.csv',
 'States/Andhra Pradesh.csv',
 'States/Manipur.csv',
 'States/Jharkhand.csv',
 'States/Tamil Nadu.csv',
 'States/Jammu and Kashmir.csv',
 'States/Assam.csv',
 'States/Rajasthan.csv',
 'States/Tripura.csv',
 'States/Karnataka.csv',
 'States/Mizoram.csv',
 'States/Puducherry.csv',
 'States/Uttar Pradesh.csv',
 'States/Sikkim.csv',
 'States/Chhattisgarh.csv',
 'States/Madhya Pradesh.csv',
 'States/Arunachal Pradesh.csv',
 'States/Odisha.csv',
 'States/Chandigarh.csv',
 'States/Goa.csv',
 'States/Meghalaya.csv',
 'States/Gujarat.csv',
 'States/Lakshadweep.csv',
 'States/Kerala.csv',
 'States/Telangana.csv',
 'States/Andaman and Nicobar Islands.csv',
 'States/Haryana.csv',
 'States/Ladakh.csv',
 'States/Maharashtra.csv',
 'States/Himachal Pradesh.csv',
 'States/West Bengal.csv',
 'States/Bihar.csv']

In [81]:
completeDf = pd.DataFrame(columns = ['#', 'Code', 'District', 'Headquarters', 'Population', 'Area (km2)', 'Density (/km2)', 'State/UnionTerritory', 'City Tier'])
completeDf

,#,Code,District,Headquarters,Population,Area (km2),Density (/km2),State/UnionTerritory,City Tier


In [82]:
for i in x:
    df = pd.read_csv(i)
    completeDf = pd.concat([completeDf, df], ignore_index=True)

/var/folders/hz/1tq14hps0cvgb6gmb21yyrnw0000gn/T/ipykernel_56156/3195966201.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  completeDf = pd.concat([completeDf, df], ignore_index=True)


In [83]:
completeDf = completeDf.drop(columns = '#')
completeDf

,Code,District,Headquarters,Population,Area (km2),Density (/km2),State/UnionTerritory,City Tier
0,–,Chümoukedima,Chümoukedima,125400,570.0,220,Nagaland,Tier I
1,DI,Dimapur,Dimapur,379769,926.0,410,Nagaland,Tier I
2,KI,Kiphire,Kiphire,74033,1255.0,66,Nagaland,Tier II
3,KO,Kohima,Kohima,270063,1041.0,213,Nagaland,Tier I
4,LO,Longleng,Longleng,50593,885.0,89,Nagaland,Tier II
...,...,...,...,...,...,...,...,...
783,ST,Sitamarhi,Sitamarhi,3419622,2199,1491,Bihar,Tier I
784,SW,Siwan,Siwan,3318176,2219,1495,Bihar,Tier I
785,SU,Supaul,Supaul,2228397,2410,919,Bihar,Tier I
786,VA,Vaishali,Hajipur,3495021,2036,1717,Bihar,Tier I


In [84]:
cities = completeDf['District'].values.tolist()
cities

['Chümoukedima',
 'Dimapur',
 'Kiphire',
 'Kohima',
 'Longleng',
 'Mokokchung',
 'Mon',
 'Niuland',
 'Noklak',
 'Peren',
 'Phek',
 'Shamator',
 'Tseminyü',
 'Tuensang',
 'Wokha',
 'Zunheboto',
 'Amritsar',
 'Barnala',
 'Bathinda',
 'Firozpur',
 'Faridkot',
 'Fatehgarh Sahib',
 'Fazilka',
 'Gurdaspur',
 'Hoshiarpur',
 'Jalandhar',
 'Kapurthala',
 'Ludhiana',
 'Malerkotla',
 'Mansa',
 'Moga',
 'Sri Muktsar Sahib',
 'Pathankot',
 'Patiala',
 'Rupnagar',
 'Sahibzada Ajit Singh Nagar',
 'Sangrur',
 'Shahid Bhagat Singh Nagar',
 'Tarn Taran',
 'Almora',
 'Bageshwar',
 'Chamoli',
 'Champawat',
 'Dehradun',
 'Haridwar',
 'Nainital',
 'Pauri Garhwal',
 'Pithoragarh',
 'Rudraprayag',
 'Tehri Garhwal',
 'Udham Singh Nagar',
 'Uttarkashi',
 'Daman',
 'Diu',
 'Dadra and Nagar Haveli',
 'Central Delhi',
 'East Delhi',
 'New Delhi',
 'North Delhi',
 'North East Delhi',
 'North West Delhi',
 'Shahdara district',
 'South Delhi',
 'South East Delhi',
 'South West Delhi',
 'West Delhi',
 'Alluri Sitharam

2014 HRA Classification

In [85]:
tier1Cities = ['Ahmedabad', 'Bengaluru', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai', 'Pune']
len(tier1Cities)

8

In [86]:
tier2Cities = ['Agra', 'Ajmer', 'Aligarh', 'Amravati', 'Amritsar', 'Anand', 'Asansol', 'Aurangabad', 'Bareilly', 'Belagavi', 'Brahmapur', 'Bhavnagar', 'Bhiwandi', 'Bhopal', 'Bhubaneswar', 'Bikaner', 'Bilaspur', 'Bokaro Steel City', 'Burdwan ', 'Bellary', 'Chandigarh', 'Coimbatore', 'Cuttack', 'Dahod', 'Dehradun', 'Dombivli', 'Dhanbad', 'Bhilai', 'Durgapur', 'Erode', 'Faridabad', 'Ghaziabad', 'Gorakhpur', 'Guntur', 'Gurgaon', 'Guwahati', 'Gwalior', 'Hamirpur', 'Hubballi–Dharwad', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jalgaon', 'Jammu', 'Jamshedpur', 'Jhansi', 'Jodhpur', 'Kalaburagi', 'Kakinada', 'Kannur', 'Kanpur', 'Karnal', 'Kochi', 'Kolhapur', 'Kollam', 'Kota', 'Kozhikode', 'Kumbakonam,Kurnool', 'Ludhiana', 'Lucknow', 'Madurai', 'Malappuram', 'Mathura', 'Mangaluru', 'Meerut', 'Mohali', 'Moradabad', 'Mysuru', 'Nagpur', 'Nanded', 'Nadiad', 'Nashik', 'Nellore', 'Noida', 'Patna', 'Puducherry', 'Purulia', 'Prayagraj', 'Raipur', 'Rajkot', 'Ranchi', 'Rourkela', 'Ratlam', 'Saharanpur', 'Salem', 'Sangli', 'Shimla', 'Siliguri', 'Solapur', 'Srinagar', 'Surat', 'Thanjavur', 'Thiruvananthapuram', 'Thrissur', 'Tiruchirappalli', 'Tirunelveli', 'Tiruvannamalai', 'Ujjain', 'Vijayapura', 'Vadodara', 'Varanasi', 'Vasai-Virar', 'Vijayawada', 'Visakhapatnam', 'Vellore', 'Warangal']
len(tier2Cities)

107

In [87]:
tier3Cities = list(set(cities) - set(tier1Cities + tier2Cities))
len(tier3Cities)

698

In [88]:
completeDf = completeDf.drop(columns = 'City Tier')
completeDf['City Tier'] = 'Tier III'
completeDf

,Code,District,Headquarters,Population,Area (km2),Density (/km2),State/UnionTerritory,City Tier
0,–,Chümoukedima,Chümoukedima,125400,570.0,220,Nagaland,Tier III
1,DI,Dimapur,Dimapur,379769,926.0,410,Nagaland,Tier III
2,KI,Kiphire,Kiphire,74033,1255.0,66,Nagaland,Tier III
3,KO,Kohima,Kohima,270063,1041.0,213,Nagaland,Tier III
4,LO,Longleng,Longleng,50593,885.0,89,Nagaland,Tier III
...,...,...,...,...,...,...,...,...
783,ST,Sitamarhi,Sitamarhi,3419622,2199,1491,Bihar,Tier III
784,SW,Siwan,Siwan,3318176,2219,1495,Bihar,Tier III
785,SU,Supaul,Supaul,2228397,2410,919,Bihar,Tier III
786,VA,Vaishali,Hajipur,3495021,2036,1717,Bihar,Tier III


In [95]:
completeDf['City Tier'] = completeDf['District'].apply(lambda x: 'Tier I' if x in tier1Cities else ('Tier II' if x in tier2Cities else 'Tier III'))

In [96]:
completeDf[completeDf['State/UnionTerritory'] == 'Tamil Nadu']

,Code,District,Headquarters,Population,Area (km2),Density (/km2),State/UnionTerritory,City Tier
132,AY,Ariyalur,Ariyalur,752481,3208,387,Tamil Nadu,Tier III
133,CGL,Chengalpattu,Chengalpattu,2556244,2945,868,Tamil Nadu,Tier III
134,CH,Chennai,Chennai,7139882,426,17000,Tamil Nadu,Tier I
135,CO,Coimbatore,Coimbatore,3472578,4723,748,Tamil Nadu,Tier II
136,CU,Cuddalore,Cuddalore,2600880,3999,702,Tamil Nadu,Tier III
137,DH,Dharmapuri,Dharmapuri,1502900,4532,332,Tamil Nadu,Tier III
138,DGL,Dindigul,Dindigul,2161367,6058,357,Tamil Nadu,Tier III
139,ER,Erode,Erode,2259608,5714,397,Tamil Nadu,Tier II
140,KL,Kallakurichi,Kallakurichi,1682687,3520,478,Tamil Nadu,Tier III
141,KC,Kanchipuram,Kanchipuram,1166401,1656,704,Tamil Nadu,Tier III


In [97]:
completeDf

,Code,District,Headquarters,Population,Area (km2),Density (/km2),State/UnionTerritory,City Tier
0,–,Chümoukedima,Chümoukedima,125400,570.0,220,Nagaland,Tier III
1,DI,Dimapur,Dimapur,379769,926.0,410,Nagaland,Tier III
2,KI,Kiphire,Kiphire,74033,1255.0,66,Nagaland,Tier III
3,KO,Kohima,Kohima,270063,1041.0,213,Nagaland,Tier III
4,LO,Longleng,Longleng,50593,885.0,89,Nagaland,Tier III
...,...,...,...,...,...,...,...,...
783,ST,Sitamarhi,Sitamarhi,3419622,2199,1491,Bihar,Tier III
784,SW,Siwan,Siwan,3318176,2219,1495,Bihar,Tier III
785,SU,Supaul,Supaul,2228397,2410,919,Bihar,Tier III
786,VA,Vaishali,Hajipur,3495021,2036,1717,Bihar,Tier III


In [98]:
completeDf.to_csv('cityDataStuff.csv', index = False)